In [125]:
# https://www.geeksforgeeks.org/sparse-matrix-representations-set-3-csr/
struct MyCSR
    m::Int64                  # Number of rows
    n::Int64                  # Number of columns
    rowptr::Vector{Int64}     # Row j is in rowptr[j]:(rowptr[j+1]-1)
    colval::Vector{Int64}     # Col indices of stored values
    nzval::Vector{Float64}    # Stored values, typically nonzeros
    MyCSR(m,n,rowptr,colval,nzval)=new(m,n,rowptr,colval, nzval)
end

In [126]:
import Base.*
function *(M::MyCSR, v::Vector{Float64})
    nw=zeros(length(v))
    for row in 1:M.m
        for i in M.rowptr[row]:M.rowptr[row+1]-1
            col=M.colval[i]
            nw[row]+=v[col]*M.nzval[i]
        end
    end
    return nw
end 

* (generic function with 329 methods)

In [127]:
using LinearAlgebra;
import Pkg; Pkg.add("JSON")
using JSON;

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [128]:
input = Dict()
open("links-20.json", "r") do f
    global input
    dicttxt = read(f, String)  # file information to string
    input=JSON.parse(dicttxt)  # parse and transform data
end

Dict{String, Any} with 5818 entries:
  "https://cmu.edu/news-ev… => Any[]
  "https://cmu.edu/news/st… => Any[]
  "https://cmu.edu/news/ne… => Any[]
  "https://cmu.edu/news/st… => Any[]
  "https://cmu.edu/Academi… => Any[]
  "https://cmu.edu/dietric… => Any[]
  "https://cmu.edu/news/st… => Any[]
  "https://cmu.edu/physics… => Any[]
  "https://cmu.edu/news/st… => Any[]
  "https://cmu.edu/news/st… => Any[]
  "https://cmu.edu/hr/care… => Any[]
  "https://cmu.edu/math/pe… => Any[]
  "https://cmu.edu/news/st… => Any[]
  "https://cmu.edu/news/st… => Any[]
  "https://cmu.edu/news/st… => Any["https://cmu.edu", "https://cmu.edu/news/ind…
  "https://cmu.edu/news/st… => Any[]
  "https://cmu.edu/leaders… => Any["https://cmu.edu", "https://cmu.edu/leadersh…
  "https://cmu.edu/piper/n… => Any[]
  "https://cmu.edu/news/fe… => Any[]
  "https://cmu.edu/news/st… => Any[]
  "https://cmu.edu/news/st… => Any[]
  "https://cmu.edu/news/st… => Any[]
  "https://cmu.edu/stugov/… => Any[]
  "https://cmu.edu/news/

In [129]:
damping=0.85
function get_matrix(exitLinks)
    n=length(exitLinks)
    exitCount=Dict{Int64, Int64}()
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, list) in exitLinks
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        push!(exitCount, i=>length(list))
        i+=1
    end
    
    
    enter=Vector{Vector{Int64}}()
    for _ in 1:n
        push!(enter, Vector{Int64}())
    end
    for (f, list) in exitLinks 
        from=decodeNames[f]
        for t in list 
            to=decodeNames[t]
            push!(enter[to], from)
        end
    end
    
    
    rowptr=ones(Int64, n+1)
    colval=Vector{Int64}()
    nzval=Vector{Float64}()
    i=2
    for to in 1:length(enter) 
        rowptr[i]=rowptr[i-1]+length(enter[to])
        for from in enter[to]
            append!(nzval, 1/exitCount[from])
            append!(colval, from)
        end
        i+=1
    end
    return (MyCSR(n,n, rowptr, colval, nzval), encodeNames, n)
end

get_matrix (generic function with 1 method)

In [130]:
function sparse_power(input, k, β=0.85)
    M, encode, n = get_matrix(input)
    v = ones(n)/n
    for i in 1:k
        v=β*(M*v).+(1-β)/n
        v/=sum(v)
    end
    v=map(r-> (encode[r[1]],Float64(r[2])), enumerate(v))
    return sort(v, by=x->x[2], rev=true)
end

sparse_power (generic function with 2 methods)

In [131]:
res=sparse_power(input,42)

5818-element Vector{Tuple{String, Float64}}:
 ("https://cmu.edu", 0.018021005253354185)
 ("https://cmu.edu/legal", 0.014751304954430604)
 ("https://cmu.edu/coronavirus/index.html", 0.011589885500141494)
 ("https://cmu.edu/coronavirus/health-and-wellness/facial-covering.html", 0.011250180465750074)
 ("https://cmu.edu/coronavirus/visitor-protocol/index.html", 0.011246572527832337)
 ("https://cmu.edu/coronavirus/health-and-wellness/testing/index.html", 0.011245901945699653)
 ("https://cmu.edu/coronavirus/staff/index.html", 0.011245252224426803)
 ("https://cmu.edu/coronavirus/preparing-campus/touchdown.html", 0.011245252224426803)
 ("https://cmu.edu/coronavirus/return/mitigation2.html", 0.011245252224426803)
 ("https://cmu.edu/coronavirus/health-and-wellness/coping-support.html", 0.011245252224426803)
 ("https://cmu.edu/coronavirus/return/resources.html", 0.011245252224426803)
 ("https://cmu.edu/coronavirus/visitor-protocol/covid-19-mitigation-protocol-for-cmu-service-providers.html", 0.01

In [160]:
 using Printf

In [162]:
join(map(r->"\\"*"hline\n"*r[1]*" & "*@sprintf("%1.5f", r[2])*"\\\\" ,res[1:40]))*"\\hline"

"\\hline\nhttps://cmu.edu & 0.01802\\\\\\hline\nhttps://cmu.edu/legal & 0.01475\\\\\\hline\nhttps://cmu.edu/coronavirus/index.html & 0.01159\\\\\\hline\nhttps://cmu.edu/coronavirus/health-and-wellness/facial-covering.html & 0.01125\\\\\\hline\nhttps://cmu.edu/coronavirus/visitor-protocol/index.html & 0.01125\\\\\\hline\nhttps://cmu.edu/coronavirus/health-and-wellness/testing/index.html & 0.01125\\\\\\hline\nhttps://cmu.edu/coronavirus/staff/index.html & 0.01125\\\\\\hline\nhttps://cmu.edu/coronavirus/preparing-campus/touchdown.html & 0.01125\\\\\\hline\nhttps://cmu.edu/coronavirus/return/mitigation2.html & 0.01125\\\\\\hline\nhttps://cmu.edu/coronavirus/health-and-wellness/coping-support.html & 0.01125\\\\\\hline\nhttps://cmu.edu/coronavirus/return/resources.html & 0.01125\\\\\\hline\nhttps://cmu.edu/coronavirus/visitor-protocol/covid-19-mitigation-protocol-for-cmu-service-providers.html & 0.01125\\\\\\hline\nhttps://cmu.edu/coronavirus/health-and-wellness/reporting-concerns.html & 0.0